In [ ]:
import calendar
import numpy as np
import pandas as pd
import scipy as sp
from random_address import real_random_address, real_random_address_by_state

def sample_cdf(cdf, sample_size):
    def _helper(cdf, prob):
        for i, p in enumerate(cdf):
            if prob < p:
                return i-1
    return [_helper(cdf, prob) for prob in np.random.random(size=sample_size)]

class simdata:
    def __init__(self, population_size, curr_year):
        self.curr_year = curr_year
        self.population_size = population_size

        census = pd.read_csv("./qid/census.csv")
        occupations = pd.read_csv("./qid/occupations.csv")
        marital_statuses = pd.read_csv("./qid/marital.csv")
        ethnicities = pd.read_csv("./qid/ethnicity.csv")

        tot = census["men"]+census["woman"]
        age_pdf = tot / sum(tot)
        age_cdf = [sum(age_pdf[:i+1]) for i in range(len(age_pdf.values))]
        gender_p = census["men"] / tot

        ages = sample_cdf(age_cdf, self.population_size)
        genders = ["m" if np.random.random() < gender_p[age] else "f" for age in ages]

        bday = [curr_year-age for age in ages]

        addresses = [real_random_address_by_state('32409') for _ in range(len(population_size))]
        street = [add["address1"] for add in addresses]
        city = [add["city"] for add in addresses]
        state = [add["state"] for add in addresses]
        postalCode = [add["postalCode"] for add in addresses]
        
        # occupation
        #   randomly get from occupation.csv

        # marital status
        #   uniform from csv 

        # race
        #   uniform from csv
        pass

    def add_quasiid(self, idname, func):
        # map func over data
        pass

    def define_datum(self, fit_data, sensitive_data):
        # fit_data - existing dataset to model data from and combine with quasiid
        #   diabetes
        # sensitive data - additional distributions / datasets to simulated sensitive fields
        #   categorical
        #   dlgn
        pass
        
    def trainset(self, sample_size, columns):
        # sample from population - return columns 
        pass

    def population(self):
        # get the whole population 
        pass

class privacy:
    def __init__(self):
        pass

    def idr(self, syndata, population, qids, svs):
        pass

In [ ]:
real_random_address_by_state('CA')

print([real_random_address_by_state('CA') for i in range(10)])

[{'address1': '6420 Via Baron', 'address2': '', 'city': 'Rancho Palos Verdes', 'state': 'CA', 'postalCode': '90275', 'coordinates': {'lat': 33.741162, 'lng': -118.390767}}, {'address1': '637 Britannia Drive', 'address2': '', 'city': 'Vallejo', 'state': 'CA', 'postalCode': '94591', 'coordinates': {'lat': 38.10476999999999, 'lng': -122.193849}}, {'address1': '2042 High Street', 'address2': '', 'city': 'Oakland', 'state': 'CA', 'postalCode': '94601', 'coordinates': {'lat': 37.776415, 'lng': -122.209418}}, {'address1': '61 Callahan Way', 'address2': '#B4', 'city': 'Mammoth Lakes', 'state': 'CA', 'postalCode': '93546', 'coordinates': {'lat': 37.647421, 'lng': -118.978423}}, {'address1': '1937 Carleton Street', 'address2': 'M', 'city': 'Berkeley', 'state': 'CA', 'postalCode': '94704', 'coordinates': {'lat': 37.861126, 'lng': -122.270083}}, {'address1': '2531 Prestwick Avenue', 'address2': '', 'city': 'Concord', 'state': 'CA', 'postalCode': '94519', 'coordinates': {'lat': 37.9966039, 'lng': -

In [ ]:
occupations = pd.read_csv("./qid/occupations.csv")
marital_statuses = pd.read_csv("./qid/marital.csv")
ethnicities = pd.read_csv("./qid/ethnicity.csv")



In [ ]:
# define simdata

# generate simdata

# train sdv with simdata

# compute privacy with synthetic data

# report

